# Medium-size system sweep

This experiment simulates a 2000 particle system in a $30 \times 30$ sweep over temperature values ranging from $0.1$ to $2.0$ and density values from $0.1$ to $1.4$.

Overall, we need to run 900 individual simulations, each with 2000 particles.  The full experiment takes about 2.5 hours to run on my machine, so I've put the analysis in a separate notebook, so that it can be re-run easily once we have generated the simulation data.

In [1]:
import lennardjonesium as lj

In [7]:
sc = lj.SweepConfiguration()

sc.system.temperature_start = 0.1
sc.system.temperature_stop = 2.0
sc.system.temperature_steps = 30

sc.system.density_start = 0.1
sc.system.density_stop = 1.4
sc.system.density_steps = 30

sc.system.particle_count = 2000
sc.system.cutoff_distance = 2.5

sc.templates.directory = 'data/T_{temperature:f}/d_{density:f}'

sc.equilibration.tolerance = 0.02
sc.equilibration.sample_size = 500
sc.equilibration.adjustment_interval = 500
sc.equilibration.steady_state_time = 5000
sc.equilibration.timeout = 150000

sc.observation.tolerance = 0.08
sc.observation.sample_size = 1000
sc.observation.observation_interval = 1000
sc.observation.observation_count = 10

sc.write('sweep.ini')

In [8]:
result = lj.run_sweep(
    'sweep.ini',
    thread_count=12,
    random_seed=lj.SeedGenerator().get,
)

================= Lennard-Jones Sweep Simulation ==================

Temperature: 30 points in the range [0.1, 2.0]
Density: 30 points in the range [0.1, 1.4]

Number of Particles: 2000
Time Step: 0.005

Chunk index: 0, Chunk count: 1
Running 900 jobs over 12 threads

Begin simulation sweep...
Jobs queued: 0, Running: 0, Completed: 900                
End simulation sweep

Job status:
Completed: 897
Aborted during Equilibration: 0
Aborted during Observation: 3

19927500 time steps computed using 12 threads in 8741.205 seconds
0.005 thread-seconds per step, or 5.264 milliseconds
Average framerate per thread: 189.977 fps


In [9]:
# Check to see how many observations were obtained on the runs that aborted during observation
[len(entry.event_data.observations_recorded) for entry in result.observation_aborted]

[9, 8, 6]

In [10]:
import os
_ = os.system('du -sh data')

2.9G	data


This time, the data directory is much to large.  We will exclude it entirely from `git`.